# 产生图片

In [1]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
import tensorflow.contrib.slim.nets as nets
import tempfile
import tarfile
import os
import json
import matplotlib.pyplot as plt
from urllib.request import urlretrieve
import PIL
import numpy as np
import xlwt
import cv2

tf.logging.set_verbosity(tf.logging.ERROR)
size=nets.inception.inception_v2.default_image_size
# os.environ["TF_CPP_MIN_LOG_LEVEL"]='2'
sess = tf.InteractiveSession()
imagenet_json, _ = urlretrieve(
    'http://www.anishathalye.com/media/2017/07/25/imagenet.json')
with open(imagenet_json) as f:
    imagenet_labels = json.load(f)
image = tf.Variable(tf.zeros((size, size, 3)))
data_dir = tempfile.mkdtemp()

#分类画图函数
def set_style(name,height,bold=False):
    style = xlwt.XFStyle()
    font = xlwt.Font()
    font.name = name
    font.bold = bold
    font.color_index = 4
    font.height = height
    style.font = font
    return style

def classify(img, correct_class=None, target_class=None):
#     fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 8))
#     fig.sca(ax1)
    p = sess.run(probs, feed_dict={image: img})[0]
#     ax1.imshow(img)
#     ax1.tick_params(axis='x',colors ='white')
#     ax1.tick_params(axis='y',colors ='white')
#     fig.sca(ax1)
    topk = list(p.argsort()[-10:][::-1])
#     print(p)
    topprobs = p[topk]
#     barlist = ax2.bar(range(10), topprobs)
#     if target_class in topk:
#         barlist[topk.index(target_class)].set_color('r')
#     if correct_class in topk:
#         barlist[topk.index(correct_class)].set_color('g')
#     plt.sca(ax2)
#     plt.ylim([0, 1.1])
#     ax2.tick_params(axis='y',colors ='white')
#     plt.xticks(range(10),
#                [imagenet_labels[i][:15] for i in topk],
#                rotation='vertical',color='white')
#     fig.subplots_adjust(bottom=0.2)
#     plt.show()

#几种模型

def inception_v3(image, reuse=False):
    preprocessed = tf.multiply(tf.subtract(tf.expand_dims(image, 0), 0.5), 2.0)
    arg_scope = nets.inception.inception_v3_arg_scope(weight_decay=0.0)
    
    with slim.arg_scope(arg_scope):
        logits, _ = nets.inception.inception_v3(
            preprocessed, 1001, is_training=False, reuse=reuse)
        logits = logits[:,1:] # ignore background class
        probs = tf.nn.softmax(logits) # probabilities
#         print(probs)
#加载参数
    inception_tarball='D:\\jupyter\\adversanial\\inception_v3_2016_08_28.tar.gz'
    tarfile.open(inception_tarball, 'r:gz').extractall(data_dir)
    restore_vars = [
    var for var in tf.global_variables()
    if var.name.startswith('InceptionV3/')
    ]
    saver = tf.train.Saver(restore_vars)
    saver.restore(sess, os.path.join(data_dir, 'inception_v3.ckpt'))
    return logits,probs

def inception_v1(image, reuse=False):
    preprocessed = tf.multiply(tf.subtract(tf.expand_dims(image, 0), 0.5), 2.0)
    arg_scope = nets.inception.inception_v1_arg_scope(weight_decay=0.0)
    
    with slim.arg_scope(arg_scope):
        logits, _ = nets.inception.inception_v1(
            preprocessed, 1001, is_training=False, reuse=reuse)
        logits = logits[:,1:] # ignore background class
        probs = tf.nn.softmax(logits) # probabilities
#         print(probs)
#加载参数
    inception_tarball='D:\\jupyter\\adversanial\\inception_v1_2016_08_28.tar.gz'
    tarfile.open(inception_tarball, 'r:gz').extractall(data_dir)
    restore_vars = [
    var for var in tf.global_variables()
    if var.name.startswith('InceptionV1/')
    ]
    saver = tf.train.Saver(restore_vars)
    saver.restore(sess, os.path.join(data_dir, 'inception_v1.ckpt'))
    return logits,probs

def inception_v2(image, reuse=False):
    preprocessed = tf.multiply(tf.subtract(tf.expand_dims(image, 0), 0.5), 2.0)
    arg_scope = nets.inception.inception_v2_arg_scope(weight_decay=0.0)
    
    with slim.arg_scope(arg_scope):
        logits, _ = nets.inception.inception_v2(
            preprocessed, 1001, is_training=False, reuse=reuse)
        logits = logits[:,1:] # ignore background class
        probs = tf.nn.softmax(logits) # probabilities
#         print(probs)
#加载参数
    inception_tarball='D:\\jupyter\\adversanial\\inception_v2_2016_08_28.tar.gz'
    tarfile.open(inception_tarball, 'r:gz').extractall(data_dir)
    restore_vars = [
    var for var in tf.global_variables()
    if var.name.startswith('InceptionV2/')
    ]
    saver = tf.train.Saver(restore_vars)
    saver.restore(sess, os.path.join(data_dir, 'inception_v2.ckpt'))
    return logits,probs


logits,probs=inception_v2(image)
img_class = 151
x = tf.placeholder(tf.float32, (size, size, 3))
x_hat = image # our trainable adversarial input
assign_op = tf.assign(x_hat, x)
learning_rate = tf.placeholder(tf.float32, ())
y_hat = tf.placeholder(tf.int32, ())
epsilon = tf.placeholder(tf.float32, ())
below = x - epsilon
above = x + epsilon
demo_epsilon = 8.0/255.0 # a really small perturbation
demo_lr = 1e-1
demo_steps = 100
demo_target = img_class # dog
labels = 1-tf.one_hot(y_hat, 1000)
loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=[labels])
optim_step = tf.train.GradientDescentOptimizer(
learning_rate).minimize(loss, var_list=[x_hat])
projected = tf.clip_by_value(tf.clip_by_value(x_hat, below, above), 0, 1)
with tf.control_dependencies([projected]):
    project_step = tf.assign(x_hat, projected)


# img_path, _ = urlretrieve('https://timgsa.baidu.com/timg?image&quality=80&size=b9999_10000&sec=1572254695068&di=0de174eab6020439abc940ae575819ff&imgtype=0&src=http%3A%2F%2Fdpic.tiankong.com%2F00%2Fzu%2FQJ6543291920.jpg%3Fx-oss-process%3Dstyle%2Fshows')
img_path='C:/Users/lenovouser/Desktop/test/'
f = xlwt.Workbook()
sheet1 = f.add_sheet('sheet1',cell_overwrite_ok=True)
row0 = ["原类别","未加噪声正确率",
        "最高类别","未加噪声正确率","top1类别","top1识别率","原来类识别"]
for j in range(0,len(row0)):
        sheet1.write(0,j,row0[j],set_style('Times New Roman',220,True))
for i in range(1,101):
    img = PIL.Image.open(''.join([img_path,str(i),".jpg"]))
    big_dim = max(img.width, img.height)
    wide = img.width > img.height
    new_w = size if not wide else int(img.width * size / img.height)
    new_h = size if wide else int(img.height * size / img.width)
    img = img.resize((new_w, new_h)).crop((0, 0, size, size))
    img = (np.asarray(img) / 255.0).astype(np.float32)
    sess.run(assign_op, feed_dict={x: img})
    p = sess.run(probs, feed_dict={image: img})[0]
    top = p.argsort()[-10:][::-1][0]
    row=[imagenet_labels[img_class],str(p[img_class]),imagenet_labels[top],str(p[top])]
#     classify(img, correct_class=img_class)
# projected gradient descent
    for k in range(demo_steps):
        # gradient descent step
        _, loss_value = sess.run(
            [optim_step, loss],
            feed_dict={learning_rate: demo_lr, y_hat: demo_target})
        # project step
        sess.run(project_step, feed_dict={x: img, epsilon: demo_epsilon})
#         if (i+1) % 10 == 0:
#             print('step %d, loss=%g' % (i+1, loss_value))
    adv = x_hat.eval() # retrieve the adversarial example
    p1 = sess.run(probs, feed_dict={image: adv})[0]
    top1 = p1.argsort()[-10:][::-1][0]
#     diff=adv-img
#     m=np.min(diff)
#     diff=diff-m
    adv = (np.asarray(adv) * 255.0).astype(np.uint8)
    adv=PIL.Image.fromarray(adv)
    adv.save("".join([img_path,"v2/",str(i),".jpg"]))
    row1=[imagenet_labels[top1],
          str(p1[top1]),
          str(p1[img_class])]
#     classify(img, correct_class=img_class)
    row=row+row1
    for j in range(0,len(row)):
        sheet1.write(i,j,row[j],set_style('Times New Roman',220,True))
f.save('C:/Users/lenovouser/Desktop/test/v2/test.xls')
    

# 融合

In [1]:
import xlrd
import tensorflow as tf
import tensorflow.contrib.slim as slim
import tensorflow.contrib.slim.nets as nets
import tempfile
import tarfile
import os
import json
import matplotlib.pyplot as plt
from urllib.request import urlretrieve
import PIL
import numpy as np
import xlwt
import cv2

tf.logging.set_verbosity(tf.logging.ERROR)
size=nets.inception.inception_v1.default_image_size
# os.environ["TF_CPP_MIN_LOG_LEVEL"]='2'
sess = tf.InteractiveSession()
imagenet_json, _ = urlretrieve(
    'http://www.anishathalye.com/media/2017/07/25/imagenet.json')
with open(imagenet_json) as f:
    imagenet_labels = json.load(f)
image = tf.Variable(tf.zeros((size, size, 3)))
data_dir = tempfile.mkdtemp()
def inception_v3(image, reuse=False):
    preprocessed = tf.multiply(tf.subtract(tf.expand_dims(image, 0), 0.5), 2.0)
    arg_scope = nets.inception.inception_v3_arg_scope(weight_decay=0.0)
    
    with slim.arg_scope(arg_scope):
        logits, _ = nets.inception.inception_v3(
            preprocessed, 1001, is_training=False, reuse=reuse)
        logits = logits[:,1:] # ignore background class
        probs = tf.nn.softmax(logits) # probabilities
#         print(probs)
#加载参数
    inception_tarball='D:\\jupyter\\adversanial\\inception_v3_2016_08_28.tar.gz'
    tarfile.open(inception_tarball, 'r:gz').extractall(data_dir)
    restore_vars = [
    var for var in tf.global_variables()
    if var.name.startswith('InceptionV3/')
    ]
    saver = tf.train.Saver(restore_vars)
    saver.restore(sess, os.path.join(data_dir, 'inception_v3.ckpt'))
    return logits,probs

def inception_v1(image, reuse=False):
    preprocessed = tf.multiply(tf.subtract(tf.expand_dims(image, 0), 0.5), 2.0)
    arg_scope = nets.inception.inception_v1_arg_scope(weight_decay=0.0)
    
    with slim.arg_scope(arg_scope):
        logits, _ = nets.inception.inception_v1(
            preprocessed, 1001, is_training=False, reuse=reuse)
        logits = logits[:,1:] # ignore background class
        probs = tf.nn.softmax(logits) # probabilities
#         print(probs)
#加载参数
    inception_tarball='D:\\jupyter\\adversanial\\inception_v1_2016_08_28.tar.gz'
    tarfile.open(inception_tarball, 'r:gz').extractall(data_dir)
    restore_vars = [
    var for var in tf.global_variables()
    if var.name.startswith('InceptionV1/')
    ]
    saver = tf.train.Saver(restore_vars)
    saver.restore(sess, os.path.join(data_dir, 'inception_v1.ckpt'))
    return logits,probs

def inception_v2(image, reuse=False):
    preprocessed = tf.multiply(tf.subtract(tf.expand_dims(image, 0), 0.5), 2.0)
    arg_scope = nets.inception.inception_v2_arg_scope(weight_decay=0.0)
    
    with slim.arg_scope(arg_scope):
        logits, _ = nets.inception.inception_v2(
            preprocessed, 1001, is_training=False, reuse=reuse)
        logits = logits[:,1:] # ignore background class
        probs = tf.nn.softmax(logits) # probabilities
#         print(probs)
#加载参数
    inception_tarball='D:\\jupyter\\adversanial\\inception_v2_2016_08_28.tar.gz'
    tarfile.open(inception_tarball, 'r:gz').extractall(data_dir)
    restore_vars = [
    var for var in tf.global_variables()
    if var.name.startswith('InceptionV2/')
    ]
    saver = tf.train.Saver(restore_vars)
    saver.restore(sess, os.path.join(data_dir, 'inception_v2.ckpt'))
    return logits,probs

logits,probs=inception_v1(image)
img_class = 151
x = tf.placeholder(tf.float32, (size, size, 3))
x_hat = image # our trainable adversarial input
assign_op = tf.assign(x_hat, x)
learning_rate = tf.placeholder(tf.float32, ())
y_hat = tf.placeholder(tf.int32, ())
epsilon = tf.placeholder(tf.float32, ())
below = x - epsilon
above = x + epsilon
demo_epsilon = 8.0/255.0 # a really small perturbation
demo_lr = 1e-1
demo_steps = 100
demo_target = img_class # dog
labels = tf.one_hot(y_hat, 1000)
loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=[labels])
optim_step = tf.train.GradientDescentOptimizer(
learning_rate).minimize(-loss, var_list=[x_hat])
projected = tf.clip_by_value(tf.clip_by_value(x_hat, below, above), 0, 1)
with tf.control_dependencies([projected]):
    project_step = tf.assign(x_hat, projected)


img_path='C:/Users/lenovouser/Desktop/test/'
"""
ac=[]
#获得误判率
for i in range(1,4):
    file="".join([img_path,"v",str(i),"/","test.xls"])
    wb = xlrd.open_workbook(filename=file)#打开文件
    sheet1 = wb.sheet_by_index(0)#通过索引获取表格
    cols = sheet1.col_values(6)#获取列内容
    n=[float(j) for j in cols[1:]]
    ac.append(n)

n1=-np.log(np.array(ac[0]))
n2=-np.log(np.array(ac[1]))
n3=-np.log(np.array(ac[2]))

for i in range(1,101):
    
    imgp1="".join([img_path,"v1/",str(i),".jpg"])
    imgp2="".join([img_path,"v2/",str(i),".jpg"])
    imgp3="".join([img_path,"v3/",str(i),".jpg"])
    image1=cv2.imread(imgp1)
    image2=cv2.imread(imgp2)
    image3=cv2.imread(imgp3)
    
    image1=cv2.resize(image1,(224,224),interpolation=cv2.INTER_CUBIC)
    image2=cv2.resize(image2,(224,224),interpolation=cv2.INTER_CUBIC)
    image3=cv2.resize(image3,(224,224),interpolation=cv2.INTER_CUBIC)
#     w1=n1[i-1]
#     w2=n2[i-1]
#     w3=n3[i-1]
#     img = (image1*w1+image2*w2+image2*w3)/(w1+w2+w3)
    img=(image1*1.0+image2*1.0+image3*1.0)/3.0
    img = (np.asarray(img)).astype(np.uint8)
    
    cv2.imwrite(img_path+"img/"+str(i)+".jpg",img)
"""

'\nac=[]\n#获得误判率\nfor i in range(1,4):\n    file="".join([img_path,"v",str(i),"/","test.xls"])\n    wb = xlrd.open_workbook(filename=file)#打开文件\n    sheet1 = wb.sheet_by_index(0)#通过索引获取表格\n    cols = sheet1.col_values(6)#获取列内容\n    n=[float(j) for j in cols[1:]]\n    ac.append(n)\n\nn1=-np.log(np.array(ac[0]))\nn2=-np.log(np.array(ac[1]))\nn3=-np.log(np.array(ac[2]))\n\nfor i in range(1,101):\n    \n    imgp1="".join([img_path,"v1/",str(i),".jpg"])\n    imgp2="".join([img_path,"v2/",str(i),".jpg"])\n    imgp3="".join([img_path,"v3/",str(i),".jpg"])\n    image1=cv2.imread(imgp1)\n    image2=cv2.imread(imgp2)\n    image3=cv2.imread(imgp3)\n    \n    image1=cv2.resize(image1,(224,224),interpolation=cv2.INTER_CUBIC)\n    image2=cv2.resize(image2,(224,224),interpolation=cv2.INTER_CUBIC)\n    image3=cv2.resize(image3,(224,224),interpolation=cv2.INTER_CUBIC)\n#     w1=n1[i-1]\n#     w2=n2[i-1]\n#     w3=n3[i-1]\n#     img = (image1*w1+image2*w2+image2*w3)/(w1+w2+w3)\n    img=(image1*1.0+image

# 预测

In [2]:
def set_style(name,height,bold=False):
    style = xlwt.XFStyle()
    font = xlwt.Font()
    font.name = name
    font.bold = bold
    font.color_index = 4
    font.height = height
    style.font = font
    return style

f1 = xlwt.Workbook()
sheet1 = f1.add_sheet("融合v1",cell_overwrite_ok=True)

sheet2 = f1.add_sheet('v2_v1',cell_overwrite_ok=True)

sheet3 = f1.add_sheet('v3_v1',cell_overwrite_ok=True)

row0 = ["top1类别","top1识别率","原类型识别率"]

for j in range(0,len(row0)):
        sheet1.write(0,j,row0[j],set_style('Times New Roman',220,True))
        sheet2.write(0,j,row0[j],set_style('Times New Roman',220,True))
        sheet3.write(0,j,row0[j],set_style('Times New Roman',220,True))
        
for i in range(1,101):
        imgf=PIL.Image.open(img_path+"img/"+str(i)+".jpg")
        big_dim = max(imgf.width, imgf.height)
        wide = imgf.width > imgf.height
        new_w = size if not wide else int(imgf.width * size / imgf.height)
        new_h = size if wide else int(imgf.height * size / imgf.width)
        imgf = imgf.resize((new_w, new_h)).crop((0, 0, size, size))
        imgf = (np.asarray(imgf) / 255.0).astype(np.float32)
        sess.run(assign_op, feed_dict={x: imgf})
        p = sess.run(probs, feed_dict={image: imgf})[0]
        top = p.argsort()[-10:][::-1][0]
        row=[imagenet_labels[top],str(p[top]),str(p[img_class])]
        
        imgf="".join([img_path,"v2/",str(i),".jpg"])
        imgf=PIL.Image.open(imgf)
        big_dim = max(imgf.width, imgf.height)
        wide = imgf.width > imgf.height
        new_w = size if not wide else int(imgf.width * size / imgf.height)
        new_h = size if wide else int(imgf.height * size / imgf.width)
        imgf = imgf.resize((new_w, new_h)).crop((0, 0, size, size))
        imgf = (np.asarray(imgf) / 255.0).astype(np.float32)
        sess.run(assign_op, feed_dict={x: imgf})
        p = sess.run(probs, feed_dict={image: imgf})[0]
        top = p.argsort()[-10:][::-1][0]
        top = p.argsort()[-10:][::-1][0]
        row1=[imagenet_labels[top],str(p[top]),str(p[img_class])]
        
        
        imgf="".join([img_path,"v3/",str(i),".jpg"])
        imgf=PIL.Image.open(imgf)
        big_dim = max(imgf.width, imgf.height)
        wide = imgf.width > imgf.height
        new_w = size if not wide else int(imgf.width * size / imgf.height)
        new_h = size if wide else int(imgf.height * size / imgf.width)
        imgf = imgf.resize((new_w, new_h)).crop((0, 0, size, size))
        imgf = (np.asarray(imgf) / 255.0).astype(np.float32)
        sess.run(assign_op, feed_dict={x: imgf})
        p = sess.run(probs, feed_dict={image: imgf})[0]
        top = p.argsort()[-10:][::-1][0]
        top = p.argsort()[-10:][::-1][0]
        row2=[imagenet_labels[top],str(p[top]),str(p[img_class])]
        
        for j in range(0,len(row)):
            sheet1.write(i,j,row[j],set_style('Times New Roman',220,True))
            sheet2.write(i,j,row1[j],set_style('Times New Roman',220,True))
            sheet3.write(i,j,row2[j],set_style('Times New Roman',220,True))

f1.save('C:/Users/lenovouser/Desktop/test/img/v1.xls')  

        
        


diff=adv-img
# enhance=diff-np.min(diff)
# enhance=np.int0(np.exp(80*(diff-np.min(diff))))
# plt.imshow(enhance)
# plt.imshow(diff)
plt.subplot(131),plt.title("R"),plt.imshow(diff[:,:,0],cmap="gray")
plt.colorbar(orientation='horizontal')
plt.subplot(132),plt.title("G"),plt.imshow(diff[:,:,1],cmap="gray")
plt.colorbar(orientation='horizontal')
plt.subplot(133),plt.title("B"),plt.imshow(diff[:,:,2],cmap="gray")
plt.colorbar(orientation='horizontal')
plt.show()





# Robust adversarial examples

Now, we go through a more advanced example. We follow our approach for [synthesizing robust adversarial examples](https://arxiv.org/abs/1707.07397) to find a single perturbation of our cat image that's simultaneously adversarial under some chosen distribution of transformations.  We could choose any distribution of differentiable transformations; in this post, we'll synthesize a single adversarial input that's robust to rotation by $\theta \in [-\pi/4, \pi/4]$.

Before we proceed, let's check if our previous example is still adversarial if we rotate it, say by an angle of $\theta = \pi/8$.

ex_angle = np.pi/8

angle = tf.placeholder(tf.float32, ())
rotated_image = tf.contrib.image.rotate(image, angle)
rotated_example = rotated_image.eval(feed_dict={image: adv, angle: ex_angle})
classify(rotated_example, correct_class=img_class, target_class=demo_target)


Looks like our original adversarial example is not rotation-invariant!

So, how do we make an adversarial example robust to a distribution of transformations? Given some distribution of transformations $T$, we can maximize $\mathbb{E}_{t \sim T} \log P\left(\hat{y} \mid t(\hat{\mathbf{x}})\right)$, subject to $\left\lVert \mathbf{x} - \hat{\mathbf{x}} \right\rVert_\infty \le \epsilon$. We can solve this optimization problem via projected gradient descent, noting that $\nabla \mathbb{E}_{t \sim T} \log P\left(\hat{y} \mid t(\hat{\mathbf{x}})\right)$ is $\mathbb{E}_{t \sim T} \nabla \log P\left(\hat{y} \mid t(\hat{\mathbf{x}})\right)$ and approximating with samples at each gradient descent step.

Rather than manually implementing the gradient sampling, we can use a trick to get TensorFlow to do it for us: we can model our sampling-based gradient descent as doing gradient descent over an ensemble of stochastic classifiers that randomly sample from the distribution and transform their input before classifying it.

num_samples = 10
average_loss = 0
for i in range(num_samples):
    rotated = tf.contrib.image.rotate(
        image, tf.random_uniform((), minval=-np.pi, maxval=np.pi))
    rotated_logits, _ = inception(rotated, reuse=True)
    average_loss += tf.nn.softmax_cross_entropy_with_logits(
        logits=rotated_logits, labels=labels) / num_samples

We can reuse our `assign_op` and `project_step`, though we'll have to write a new `optim_step` for this new objective.


optim_step = tf.train.GradientDescentOptimizer(
    learning_rate).minimize(-average_loss, var_list=[x_hat])

Finally, we're ready to run PGD to generate our adversarial input. As in the previous example, we'll choose "guacamole" as our target class.

demo_epsilon = 8.0/255.0 # still a pretty small perturbation
demo_lr = 2e-1
demo_steps = 300
demo_target = img_class # "guacamole"

# initialization step
sess.run(assign_op, feed_dict={x: img})

# projected gradient descent
for i in range(demo_steps):
    # gradient descent step
    _, loss_value = sess.run(
        [optim_step, average_loss],
        feed_dict={learning_rate: demo_lr, y_hat: demo_target})
    # project step
    sess.run(project_step, feed_dict={x: img, epsilon: demo_epsilon})
    if (i+1) % 50 == 0:
        print('step %d, loss=%g' % (i+1, loss_value))
    

adv_robust = x_hat.eval() # retrieve the adversarial example

This adversarial image is classified as "guacamole" with high confidence, even when it's rotated!

rotated_example = rotated_image.eval(feed_dict={image: adv_robust, angle: ex_angle})
classify(rotated_example, correct_class=img_class, target_class=demo_target)

diff=adv_robust-img
# enhance=np.int0(np.exp(80*(diff-np.min(diff))))
# plt.imshow(enhance)
# plt.imshow(diff)
plt.subplot(131),plt.title("R"),plt.imshow(diff[:,:,0],cmap="gray")
plt.colorbar(orientation='horizontal')
plt.subplot(132),plt.title("G"),plt.imshow(diff[:,:,1],cmap="gray")
plt.colorbar(orientation='horizontal')
plt.subplot(133),plt.title("B"),plt.imshow(diff[:,:,2],cmap="gray")
plt.colorbar(orientation='horizontal')
plt.show()

## Evaluation

Let's examine the rotation-invariance of the robust adversarial example we produced over the entire range of angles, looking at $P(\hat{y} \mid \hat{\mathbf{x}})$ over $\theta \in [-\pi/4, \pi/4]$.

thetas = np.linspace(-np.pi, np.pi, 301)/np.pi*180

p_naive = []
p_robust = []
for theta in thetas:
    rotated = rotated_image.eval(feed_dict={image: adv_robust, angle: theta})
    p_robust.append(probs.eval(feed_dict={image: rotated})[0][demo_target])
    
    rotated = rotated_image.eval(feed_dict={image: adv, angle: theta})
    p_naive.append(probs.eval(feed_dict={image: rotated})[0][demo_target])

robust_line, = plt.plot(thetas, p_robust, color='b', linewidth=2, label='robust')
naive_line, = plt.plot(thetas, p_naive, color='r', linewidth=2, label='naive')
plt.ylim([0, 1.05])
plt.xlabel('rotation angle')
plt.ylabel('target class probability')
plt.legend(handles=[robust_line, naive_line], loc='lower right')
plt.show()

It's super effective!